In [1]:
from dgp import *

# 1. Simulate or load data
sample_size = 800
min_degree = 2
max_degree = 4
adj = get_graph(sample_size, min_degree, max_degree, seed=2)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=30,
    burnin_R=10, seed=1)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 40/40 [00:01<00:00, 30.66it/s]


In [3]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=10,
    seed=0
)

{'average': 0.28174999999999994,
 'direct_effect': -0.14290000000000003,
 'spillover_effect': -0.11787499999999995,
 'psi_1_gamma': 0.23559999999999998,
 'psi_0_gamma': 0.3785,
 'psi_zero': 0.49637499999999996}

In [2]:
from drnet import doubly_robust
from run_pll import cols_raw

ret_i = doubly_robust(A_chain[-2], L_chain[-2], Y_chain[-2], adj, return_raw=False)
ret_i

{'average': 0.2829604563257415,
 'direct_effect': -0.09970894099130528,
 'spillover_effect': -0.11255021946342691,
 'psi_1_gamma': 0.2515737968274997,
 'psi_0_gamma': 0.351282737818805,
 'psi_zero': 0.4638329572822319}

In [4]:
from drnet_em import doubly_robust_em

ret_i_em = doubly_robust_em(A_chain[-3], L_chain[-3], Y_chain[-3], adj, return_raw=False)
ret_i_em

{'average': 0.2487969941961577,
 'direct_effect': -0.08784745220790113,
 'spillover_effect': -0.21207985933017354,
 'psi_1_gamma': 0.21936786082727183,
 'psi_0_gamma': 0.30721531303517297,
 'psi_zero': 0.5192951723653465}

In [ ]:
from autognet import fit_autog_models, extract_parameters_from_autog_models, prepare_features_outcome_model
import statsmodels.api as sm

def get_beta(adj_matrix, Y, A, L, treatment_allocation=0.5, R=10, burnin=5, seed=0):
    """
    Fit autognet models and evaluate causal effects using agc_effect.
    """
    np.random.seed(seed)
    models = fit_autog_models(Y, A, L, adj_matrix)
    tau, rho, nu, beta = extract_parameters_from_autog_models(models, adj_matrix)

    return beta

In [ ]:
from tqdm import tqdm

betahats = []
for i in tqdm(range(len(Y_chain))):
    betahat = get_beta(adj, Y_chain[i], A_chain[i], L_chain[i], treatment_allocation=0.7, R=50, burnin=10, seed=1)
    betahats.append(betahat)

100%|██████████| 1000/1000 [00:17<00:00, 56.53it/s]


In [ ]:
betahats[-1]

array([-0.39173937, -0.52787214,  0.15045679,  0.00440145, -0.08004167,
       -0.40050773, -0.05185024,  0.63315126,  0.14825143, -0.201992  ])

In [ ]:
beta

array([-0.3 , -0.6 , -0.2 , -0.2 , -0.05, -0.1 , -0.01,  0.4 ,  0.01,
        0.2 ])

In [ ]:
np.mean(np.array(betahats), axis=0) - beta

array([ 0.08010531, -0.0145634 , -0.00379152,  0.00883194, -0.00116777,
        0.00224138, -0.00691273,  0.00442347, -0.00247561, -0.21096924])

In [ ]:
np.mean(np.mean(np.array(betahats), axis=0) - beta)

-0.014427816936012338

In [ ]:
num_sample = 30
num_burn = 10
sample_size = 800

from dgp import *

# 1. Simulate or load data
min_degree = 5
max_degree = 7
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain2(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 40/40 [00:01<00:00, 27.37it/s]


In [2]:
from agcEffect import *

agc_effect2(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=10,
    seed=0
)

{'average': 0.201,
 'direct_effect': 0.025250000000000022,
 'spillover_effect': -0.23697500000000005,
 'psi_1_gamma': 0.21157499999999999,
 'psi_0_gamma': 0.18632499999999996,
 'psi_zero': 0.4233}

In [13]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(0.009349342448493666, 0.4965)

In [2]:
n_cpu = 10

In [5]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x10808c1f0> in 10 tasks, with 10 processes...
Multiprocessing finished.


In [6]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.18595 ± 0.03886
direct_effect: -0.00959 ± 0.04825
spillover_effect: -0.24056 ± 0.08832
psi_0_gamma: 0.19490 ± 0.05247
psi_zero: 0.43545 ± 0.06247
psi_1_gamma: 0.18530 ± 0.03915


In [7]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x164620a60> in 10 tasks, with 10 processes...
Multiprocessing finished.


In [8]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.17153 ± 0.03228
direct_effect: -0.02476 ± 0.02289
spillover_effect: -0.20472 ± 0.06280
psi_0_gamma: 0.19140 ± 0.03556
psi_zero: 0.39611 ± 0.03821
psi_1_gamma: 0.16664 ± 0.03403


In [3]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr_em = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr_em = np.array(res_list_dr_em)

Multiprocessing <function run_dr_em at 0x15c9e0ee0> in 10 tasks, with 10 processes...
Multiprocessing finished.


In [4]:
from run_pll import column_names

ret_mean_dr_em = res_list_array_dr_em.mean(axis=0)
ret_std_dr_em = res_list_array_dr_em.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr_em[i]:.5f} ± {ret_std_dr_em[i]:.5f}")

average: 0.10687 ± 0.01503
direct_effect: 0.03032 ± 0.02366
spillover_effect: -0.19783 ± 0.07553
psi_0_gamma: 0.08564 ± 0.01672
psi_zero: 0.28347 ± 0.06859
psi_1_gamma: 0.11596 ± 0.01925
